In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras import layers,models
import os
training_path = os.path.join("Transfer_Learning_Dataset/training/training")
validation_path = os.path.join("Transfer_Learning_Dataset/validation/validation")
path = os.listdir(training_path)
training_generator = ImageDataGenerator(
    horizontal_flip = True,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    rescale = 1./255.,
    rotation_range = 20,
    )
train_data = training_generator.flow_from_directory(
    training_path, batch_size = 32, target_size=(150, 150), class_mode = "categorical", shuffle = True)

Found 1098 images belonging to 10 classes.


In [13]:
val_gen = ImageDataGenerator(
    rescale = 1./255.
)
val_data = val_gen.flow_from_directory(validation_path, batch_size = 1, target_size = (150,150),class_mode = "categorical", shuffle = True)

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image


Found 272 images belonging to 10 classes.


In [14]:
model = InceptionV3(weights='imagenet', include_top=False, input_shape = (150,150,3))
model.summary()

2021-12-05 01:06:47.406079: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [15]:
model.trainable = False
add_model = models.Sequential()
add_model.add(model)
add_model.add(layers.GlobalAveragePooling2D())
add_model.add(layers.Dropout(0.5))
add_model.add(layers.Dense(10, 
                    activation='softmax'))

In [16]:
add_model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=['accuracy'])
add_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________


In [19]:
history = add_model.fit(train_data, 
                              epochs=10,validation_data=val_data)

Epoch 1/10
35/35 [==============================] - 98s 3s/step - loss: 0.5591 - accuracy: 0.8625 - val_loss: 0.2277 - val_accuracy: 0.9191
Epoch 2/10
35/35 [==============================] - 100s 3s/step - loss: 0.4876 - accuracy: 0.8607 - val_loss: 0.2224 - val_accuracy: 0.9338
Epoch 3/10
35/35 [==============================] - 100s 3s/step - loss: 0.4597 - accuracy: 0.8698 - val_loss: 0.2980 - val_accuracy: 0.9228
Epoch 4/10
35/35 [==============================] - 99s 3s/step - loss: 0.4267 - accuracy: 0.8898 - val_loss: 0.2481 - val_accuracy: 0.9228
Epoch 5/10
35/35 [==============================] - 101s 3s/step - loss: 0.4251 - accuracy: 0.8834 - val_loss: 0.2190 - val_accuracy: 0.9338
Epoch 6/10
35/35 [==============================] - 101s 3s/step - loss: 0.4592 - accuracy: 0.8707 - val_loss: 0.2528 - val_accuracy: 0.9301
Epoch 7/10
35/35 [==============================] - 99s 3s/step - loss: 0.4436 - accuracy: 0.8852 - val_loss: 0.2262 - val_accuracy: 0.9265
Epoch 8/10
24/35

In [ ]:
add_model.evaluate(val_data)

272/272 [==============================] - 24s 78ms/step - loss: 0.2666 - accuracy: 0.9118


[0.2666400372982025, 0.9117646813392639]

In [ ]:
import random
y_test = np.zeros((500,10))
x_test = []
image_names = []

class_dict = {"n0":0,"n1":1,"n2":2,"n3":3,"n4":4,"n5":5,"n6":6,"n7":7,"n8":8,"n9":9}
for clas in path:
    for i in range(50):
        images = os.listdir(training_path+"/"+clas)
        random_name = random.choice(images)
        img = image.load_img(os.path.join(training_path+"/"+clas+"/"+random_name), target_size=(150, 150) )
        image_names.append(training_path+"/"+clas+"/"+random_name)
        x_test.append(np.array(img))
        y_test[(class_dict[clas]*50)+i][class_dict[clas]] = 1
    
    
x_test = np.asarray(x_test)

In [ ]:
x_test = x_test/255.
res = add_model.predict_classes(x_test)
actual = np.argmax(y_test, axis = -1)
pred= {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}
for i,j in zip(actual,res):
    if i == j:
        pred[i] = pred[i]+1

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
pred

{0: 0, 1: 0, 2: 0, 3: 2, 4: 2, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

In [ ]:
reverse_dict = {0:"mantled_howler",1:"patas_monkey",2:"bald_uakari",3:"japanese_macaque",4:"pygmy_marmoset",5:"white_headed_capuchin",6:"silvery_marmoset",7:"common_squirrel_monkey",8:"black_headed_night_monkey",9:"nilgiri_langur"}

In [ ]:
print("Image Name              \t    Predicted Monkey Name    \t Actual Monkey Name")
for i,j in enumerate(image_names):
    print(j,"\t",reverse_dict[res[i]],"\t",reverse_dict[actual[i]])

Image Name              	    Predicted Monkey Name    	 Actual Monkey Name
../input/10-monkey-species/training/training/n8/n8040.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8033.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8077.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8124.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8077.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8043.jpg 	 pygmy_marmoset 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8046.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8133.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/10-monkey-species/training/training/n8/n8128.jpg 	 black_headed_night_monkey 	 mantled_howler
../input/